In [1]:
import anatomist.api as anatomist
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

In [2]:
from soma import aims
import json
import sys
import pandas as pd
import scipy
import scipy.stats
import numpy as np
import glob

In [3]:
%matplotlib qt5
a = anatomist.Anatomist()

Starting Anatomist.....
global modules: /neurospin/dico/rmenasria/Runs/03_main/Program/main_pixi/.pixi/envs/default/share/anatomist-5.2/python_plugins
home   modules: /home/rm283713/.anatomist/python_plugins
config file : /home/rm283713/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
loading module gltf_io
loading module palettecontrols
loading module paletteViewer
loading module meshsplit
loading module profilewindow
loading module ana_image_math
loading module anacontrolmenu
loading module foldsplit
loading module modelGraphs
loading module bsa_proba
loading module histogram
loading module gradientpalette
loading module infowindow
loading module simple_controls
loading module volumepalettes
loading module statsplotwindow
loading module save_resampled
loading module valuesplotwindow
loading module selection
all python modules loaded
Anatomist started.


In [4]:
Rspam_model = "/neurospin/dico/rmenasria/Runs/01_essai/Program/env_pixi/.pixi/envs/default/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_right/meshes/Rspam_model_meshes_1.arg"

Lspam_model = "/neurospin/dico/rmenasria/Runs/01_essai/Program/env_pixi/.pixi/envs/default/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_left/meshes/Lspam_model_meshes_1.arg"


json_regions = "/neurospin/dico/data/deep_folding/current/sulci_regions_gridsearch.json"

path_summary = ""
path_file = "/neurospin/dico/rmenasria/Runs/03_main/Output/csv/all_regions_gravity_center_shifts_utrecht_pclean.csv"

In [5]:
with open(json_regions) as f:
    regions = json.load(f)

In [6]:
file_to_display = f"{path_summary}/{path_file}"

In [7]:
next(iter(regions['brain']))

'S.C.-sylv._left'

In [8]:
param = "distance"

In [9]:
df = pd.read_csv(file_to_display)[["region", param]]

In [10]:
res = df.groupby(['region']).mean()
res["side"] = res.index.str.split('_').str[-1]
res = res.reset_index()

In [11]:
res = res[~res.region.str.contains("fronto-parietal")]
res = res[~res.region.str.contains("STs-SGSM")]
res = res[~res.region.str.contains("FCLp-SGSM")]
res = res[~res.region.str.contains("CINGULATE")]

In [12]:
"CINGULATE_left".replace("CINGULATE", "CINGULATE.")

'CINGULATE._left'

In [13]:
def get_sulci(region):
    region = region.replace("CINGULATE", "CINGULATE.")
    region = region.replace("ORBITAL", "S.Or.")
    list_sulci = list(regions['brain'][f"{region}"].keys())
    list_sulci = [x.replace("paracingular.", "S.F.int.") for x in list_sulci]
    return list_sulci

In [14]:
get_sulci("CINGULATE_left")

['S.F.int._left', 'F.C.M.ant._left']

In [15]:
res["region"].unique()

array(['F.C.L.p.-S.GSM._left', 'F.C.L.p.-S.GSM._right',
       'F.C.L.p.-subsc.-F.C.L.a.-INSULA._left',
       'F.C.L.p.-subsc.-F.C.L.a.-INSULA._right',
       'F.C.M.post.-S.p.C._left', 'F.C.M.post.-S.p.C._right',
       'F.Coll.-S.Rh._left', 'F.Coll.-S.Rh._right',
       'F.I.P.-F.I.P.Po.C.inf._left', 'F.I.P.-F.I.P.Po.C.inf._right',
       'F.I.P._left', 'F.I.P._right', 'F.P.O.-S.Cu.-Sc.Cal._left',
       'F.P.O.-S.Cu.-Sc.Cal._right', 'Lobule_parietal_sup._left',
       'Lobule_parietal_sup._right', 'OCCIPITAL_left', 'OCCIPITAL_right',
       'S.C.-S.Pe.C._left', 'S.C.-S.Pe.C._right', 'S.C.-S.Po.C._left',
       'S.C.-S.Po.C._right', 'S.C.-sylv._left', 'S.C.-sylv._right',
       'S.F.inf.-BROCA-S.Pe.C.inf._left',
       'S.F.inf.-BROCA-S.Pe.C.inf._right', 'S.F.int.-F.C.M.ant._left',
       'S.F.int.-F.C.M.ant._right', 'S.F.int.-S.R._left',
       'S.F.int.-S.R._right', 'S.F.inter.-S.F.sup._left',
       'S.F.inter.-S.F.sup._right', 'S.F.marginal-S.F.inf.ant._left',
       'S.F.margin

In [16]:
dic = {
    "FCLp-subsc-FCLa-INSULA": "F.C.L.p.-subsc.-F.C.L.a.-INSULA.",
    "FCMpost-SpC": "F.C.M.post.-S.p.C.",
    "FColl-SRh": "F.Coll.-S.Rh.",
    "FIP": "F.I.P.",
    "FPO-SCu-ScCal": "F.P.O.-S.Cu.-Sc.Cal.",
    "Lobule_parietal_sup": "Lobule_parietal_sup",
    "SOr": "S.Or.",
    "SOlf": "S.Olf.",
    "SC-SPeC": "S.C.-S.Pe.C.",
    "SC-SPoC": "S.C.-S.Po.C.",
    "SC-sylv": "S.C.-sylv.",
    "SFinf-BROCA-SPeCinf": "S.F.inf.-BROCA-S.Pe.C.inf.",
    "SFint-FCMant": "S.F.int.-F.C.M.ant.",
    "SFint-SR": "S.F.int.-S.R.",
    "SFinter-SFsup": "S.F.inter.-S.F.sup.",
    "SFmarginal-SFinfant":"S.F.marginal-S.F.inf.ant.",
    "SFmedian-SFpoltr-SFsup": "S.F.median-S.F.pol.tr.-S.F.sup.",
    "SOr-SOlf": "S.Or.-S.Olf.",
    "SPeC": "S.Pe.C.",
    "SPoC": "S.Po.C.",
    "STi-SOTlat": "S.T.i.-S.O.T.lat.",
    "STi-STs-STpol": "S.T.i.-S.T.s.-S.T.pol.",
    "STsbr": "S.T.s.br.",
    "STs": "S.T.s.",
    "ScCal-SLi": "Sc.Cal.-S.Li.",
    "SsP-SPaint": "S.s.P.-S.Pa.int.",
}

In [17]:
for key in dic.keys():
    res["region"] = res["region"].str.replace(key, dic[key])
res.head()

,region,distance,side
2,F.C.L.p.-S.GSM._left,13.236754,left
3,F.C.L.p.-S.GSM._right,12.922353,right
4,F.C.L.p.-subsc.-F.C.L.a.-INSULA._left,11.076525,left
5,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,9.522565,right
6,F.C.M.post.-S.p.C._left,2.456146,left


In [18]:
res['sulcus'] = res.apply(lambda x: get_sulci(x.region), axis=1)
res.head()

,region,distance,side,sulcus
2,F.C.L.p.-S.GSM._left,13.236754,left,"[F.C.L.p._left, S.GSM._left]"
3,F.C.L.p.-S.GSM._right,12.922353,right,"[F.C.L.p._right, S.GSM._right]"
4,F.C.L.p.-subsc.-F.C.L.a.-INSULA._left,11.076525,left,"[F.C.L.p._left, F.C.L.r.sc.ant._left, F.C.L.r...."
5,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,9.522565,right,"[F.C.L.p._right, F.C.L.r.sc.ant._right, F.C.L...."
6,F.C.M.post.-S.p.C._left,2.456146,left,"[F.C.M.post._left, S.C.LPC._left, S.p.C._left]"


In [19]:
res = res.sort_values(by=param, ascending=False)
res = res.explode("sulcus")
res[res.region.str.contains("S.T.s.")]

,region,distance,side,sulcus
53,S.T.s.-S.GSM._right,16.925855,right,S.T.s._right
53,S.T.s.-S.GSM._right,16.925855,right,S.GSM._right
55,S.T.s._right,16.925855,right,S.T.s._right
50,S.T.i.-S.T.s.-S.T.pol._left,16.880930,left,S.T.i.ant._left
50,S.T.i.-S.T.s.-S.T.pol._left,16.880930,left,S.T.i.post._left
50,S.T.i.-S.T.s.-S.T.pol._left,16.880930,left,S.T.s._left
50,S.T.i.-S.T.s.-S.T.pol._left,16.880930,left,S.T.pol._left
52,S.T.s.-S.GSM._left,16.476346,left,S.T.s._left
52,S.T.s.-S.GSM._left,16.476346,left,S.GSM._left
54,S.T.s._left,16.476346,left,S.T.s._left


In [20]:
for _, row in res.iterrows():
    print(row.sulcus)

S.T.s._right
S.GSM._right
S.T.s._right
S.T.i.ant._left
S.T.i.post._left
S.T.s._left
S.T.pol._left
S.T.s._left
S.GSM._left
S.T.s._left
F.Coll._left
S.Rh._left
S.Rh._left
F.Coll._left
S.O.T.lat.ant._left
S.O.T.lat.int._left
S.O.T.lat.med._left
S.O.T.lat.post._left
S.T.pol._left
S.Rh._left
F.Coll._right
S.O.T.lat.ant._right
S.O.T.lat.int._right
S.O.T.lat.med._right
S.O.T.lat.post._right
S.T.pol._right
S.T.i.ant._right
S.T.i.post._right
S.T.s._right
S.T.pol._right
F.Coll._right
S.Rh._right
F.C.L.p._left
S.GSM._left
S.Rh._right
F.Coll._right
S.O.T.lat.ant._right
S.O.T.lat.int._right
S.O.T.lat.med._right
S.O.T.lat.post._right
S.Rh._left
F.Coll._left
S.O.T.lat.ant._left
S.O.T.lat.int._left
S.O.T.lat.med._left
S.O.T.lat.post._left
F.C.L.p._right
S.GSM._right
S.T.i.ant._left
S.T.i.post._left
S.O.T.lat.ant._left
S.O.T.lat.int._left
S.O.T.lat.med._left
S.O.T.lat.post._left
S.F.int._right
S.R.inf._right
S.R.sup._right
F.C.L.p._left
F.C.L.r.sc.ant._left
F.C.L.r.sc.post._left
INSULA_left
F.C.L.a._le

In [21]:
res[res.sulcus=="S.F.orbitaire._right"]

,region,distance,side,sulcus
37,S.F.marginal-S.F.inf.ant._right,7.62349,right,S.F.orbitaire._right


In [22]:
res

,region,distance,side,sulcus
53,S.T.s.-S.GSM._right,16.925855,right,S.T.s._right
53,S.T.s.-S.GSM._right,16.925855,right,S.GSM._right
55,S.T.s._right,16.925855,right,S.T.s._right
50,S.T.i.-S.T.s.-S.T.pol._left,16.880930,left,S.T.i.ant._left
50,S.T.i.-S.T.s.-S.T.pol._left,16.880930,left,S.T.i.post._left
...,...,...,...,...
58,S.s.P.-S.Pa.int._left,1.908432,left,S.s.P._left
58,S.s.P.-S.Pa.int._left,1.908432,left,S.Pa.int._left
7,F.C.M.post.-S.p.C._right,1.248387,right,F.C.M.post._right
7,F.C.M.post.-S.p.C._right,1.248387,right,S.C.LPC._right


In [23]:
df_right = res[res.side == "right"]
print(df_right.max())

region      deMatos_polar._right
distance               16.925855
side                       right
sulcus              S.s.P._right
dtype: object


In [24]:
min_shift = None
max_shift = None

def set_color_property(res, side):
    global dic
    global min_shift, max_shift
    min_shift = res[param].min()
    max_shift = res[param].max()
    print(f"min {min_shift}, max {max_shift}")

    if side == "L":
        spam_model_file = Lspam_model
    else:
        spam_model_file = Rspam_model
        
    dic[f"aims{side}"] = aims.read(spam_model_file)

    for vertex in dic[f"aims{side}"].vertices():
        vertex['p_value'] = 0.

    for _, row in res.iterrows():
        for vertex in dic[f"aims{side}"].vertices():
            vname = vertex.get('name')
            if vname == row.sulcus:
                    vertex['p_value'] = row[param]
                    # if row.p < -np.log10(0.05/56):
                    #     vertex['p_value'] = 0.
                    # elif vertex['p_value'] != 0.: 
                    #     vertex['p_value'] = min(vertex['p_value'],
                    #                             row.p)
                    # else:
                    #     vertex['p_value'] = row.p
    
    dic[f"ana{side}"] = a.toAObject(dic[f"aims{side}"])

    dic[f"ana{side}"].setColorMode(dic[f"ana{side}"].PropertyMap)
    dic[f"ana{side}"].setColorProperty('p_value')
    dic[f"ana{side}"].notifyObservers()
    
                
def visualize_whole_hemisphere(view_quaternion, side, i):
    global block
    global dic
    global min_shift
    global max_shift
    print(min_shift, max_shift)
    threshold = max(abs(min_shift), abs(max_shift))
    try:
        block
    except NameError:
        block = a.createWindowsBlock(4)

    dic[f"win{i}"] = a.createWindow('3D',
                                    block=block,
                                    no_decoration=True,
                                    options={'hidden': 1})
    dic[f"win{i}"].addObjects(dic[f"ana{side}"])

    # #fix the palette to the full shift_x range
    # dic[f"ana{side}"].setPalette(
    #     "bwr",
    #     minVal=-threshold,
    #     maxVal=threshold,
    #     absoluteMode=True
    # )

    dic[f"ana{side}"].setPalette("green_yellow_red",
                            minVal=0, maxVal=threshold,
                            absoluteMode=True)
    
    dic[f"win{i}"].camera(view_quaternion=view_quaternion)



middle_view = [0.5, -0.5, -0.5, 0.5]
side_view = [0.5, 0.5, 0.5, 0.5]
bottom_view = [0, -1, 0, 0]
top_view = [0, 0, 0, -1]
 
def visualize_whole(res, side, start):
    set_color_property(res, side)
    visualize_whole_hemisphere(middle_view if side == "L" else side_view, side, start+0)
    visualize_whole_hemisphere(top_view, side, start+1)
    visualize_whole_hemisphere(bottom_view, side, start+2)
    visualize_whole_hemisphere(side_view if side == "L" else middle_view, side, start+3)

In [25]:
dic = {}

visualize_whole(res, "L", 0)
visualize_whole(res, "R", 4)

min 1.248387075723171, max 16.925855395058882
Reading FGraph version 2.0
bounding box found : -90, -80, -90
                     90, 120, 60
1.248387075723171 16.925855395058882
1.248387075723171 16.925855395058882
1.248387075723171 16.925855395058882
1.248387075723171 16.925855395058882
min 1.248387075723171, max 16.925855395058882
Reading FGraph version 2.0
bounding box found : -90, -80, -90
                     90, 120, 60
1.248387075723171 16.925855395058882
1.248387075723171 16.925855395058882
1.248387075723171 16.925855395058882
1.248387075723171 16.925855395058882


Multitexturing present
function glActiveTexture found.
function glClientActiveTexture found.
function glBlendEquation found.
function glTexImage3D found.
function glMultiTexCoord3f found.
function glBindFramebuffer found.
function glBindRenderbuffer found.
function glFramebufferTexture2D found.
function glGenFramebuffers found.
function glGenRenderbuffers found.
function glFramebufferRenderbuffer found.
function glRenderbufferStorage found.
function glCheckFramebufferStatus found.
function glDeleteRenderbuffers found.
function glDeleteFramebuffers found.
Number of texture units: 8
function glUniform1f found.
function glUniform1i found.
function glUniform4fv found.
function glGetUniformLocation found.
function glMultTransposeMatrixf found.
function glAttachShader found.
function glDetachShader found.
function glCompileShader found.
function glCreateProgram found.
function glCreateShader found.
function glDeleteProgram found.
function glDeleteShader found.
function glGetProgramiv found.


: 